# Support Vector Machines (SVM) Model Analysis Walkthrough (Low Gravity)

Support Vector Machines (SVM) are supervised machine learning models that aim to find the hyperplane that best divides a dataset into classes. The core concept is to maximize the margin between the classes. The SVM searches for the closest points (support vectors) and attempts to set the boundary in the middle of them. SVMs can also be used for regression tasks.

In [1]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, confusion_matrix, classification_report, make_scorer, recall_score, precision_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import SMOTE, RandomOverSampler
from pathlib import Path

In [2]:
# this is the path to your pickle file (should be the same location as CSVs)
path = Path('../data')

with open(path / 'low_g_pandas_2.0.2.pkl', 'rb') as file:
    df = pickle.load(file)
df = df.reset_index(drop=True)

## Dataset Splitting
We begin by splitting our dataset into a training and testing set. This process ensures that we have a distinct set of data to evaluate the model's performance.

In [3]:
# define independent variables and dependent variable
maneuver = 'LOW-G'
X = df.drop(maneuver, axis=1)
y = df[maneuver]

In [4]:
# splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

## Build and Fit the Model
We will set up a parameter grid with the best parameters. These parameters were developed in the modeling directory in the decision tree (LOW-G) file. For more information on the training and tuning of this model please refer to the modeling file.

In [5]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

In [6]:
# parameter grid for finding the best hyperparameters
params = {
    'svm__gamma': [1],
    'svm__C': [1000],
    'svm__kernel': ['rbf'],
    'svm__class_weight': ['balanced'],
    'svm__random_state': [42]
}

In [7]:
# grid search with cross-validation
f1_scorer = make_scorer(f1_score)
strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# instantiate the grid search loading pipeline, parameters, k-fold, and scorer
grid_search = GridSearchCV(estimator=pipe, param_grid=params, cv=strat_k_fold, scoring=f1_scorer)
# fit the grid search
grid_search.fit(X_train, y_train)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svm', SVC())]),
             param_grid={'svm__C': [1000], 'svm__class_weight': ['balanced'],
                         'svm__gamma': [1], 'svm__kernel': ['rbf'],
                         'svm__random_state': [42]},
             scoring=make_scorer(f1_score))

## Make Predictions for Model Evaluation

In [8]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     1.0000    0.9999    1.0000     51437
           1     0.9914    1.0000    0.9957       344

    accuracy                         0.9999     51781
   macro avg     0.9957    1.0000    0.9978     51781
weighted avg     0.9999    0.9999    0.9999     51781



In [9]:
print(confusion_matrix(y_test, y_pred))

[[51434     3]
 [    0   344]]


## Visualizations
These visualizations will provide a comprehensive understanding of the model's performance, the features that drive decisions, and how the model's performance evolves as more data is added.

### Exploring C Values
The 'C' parameter in SVMs determines the trade-off between achieving a low error on the training data and maximizing the margin between classes. Let's see how different values of 'C' impact our model's performance.

In [10]:
# C Values

# define lists to collect scores
train_f1_scores, test_f1_scores = list(), list()
train_recall_scores, test_recall_scores = list(), list()
train_prec_scores, test_prec_scores = list(), list()

# define list of C values to plot
c_values = [0.1, 1, 10, 100, 1000]

for c in c_values:
    # configure the model
    model = SVC(kernel='rbf', gamma=1, C=c, class_weight='balanced', random_state=42)
    # fit the training dataset
    model.fit(X_train, y_train.values.ravel())
    # evaluate training dataset
    train_yhat = model.predict(X_train)
    train_f1 = f1_score(y_train, train_yhat)
    train_recall = recall_score(y_train, train_yhat)
    train_prec = precision_score(y_train, train_yhat)
    train_f1_scores.append(train_f1)
    print(train_f1_scores)
    train_recall_scores.append(train_recall)
    print(train_recall_scores)
    train_prec_scores.append(train_prec)
    print(train_prec_scores)
    # evaluate test dataset
    test_yhat = model.predict(X_test)
    test_f1 = f1_score(y_test, test_yhat)
    test_recall = recall_score(y_test, test_yhat)
    test_prec = precision_score(y_test, test_yhat)
    test_f1_scores.append(test_f1)
    print(test_f1_scores)
    test_recall_scores.append(test_recall)
    print(test_recall_scores)
    test_prec_scores.append(test_prec)
    print(test_prec_scores)    
    # summarize progress
    print('f1: >%d, train: %.3f, test: %.3f' % (c, train_f1, test_f1))
    print('recall: >%d, train: %.3f, test: %.3f' % (c, train_recall, test_recall))
    print('precision: >%d, train: %.3f, test: %.3f' % (c, train_prec, test_prec))

train_f1_scores
test_f1_scores

# plot of train and test scores vs tree depth
plt.plot(c_values, test_f1_scores, '-o', label='f1 score')
plt.plot(c_values, test_recall_scores, '-o', label='recall')
plt.plot(c_values, test_prec_scores, '-o', label='precision')
plt.title("C Values")
plt.legend()
plt.xlabel("C Value (Gamma = 1)")
plt.show()

[1.0]
[1.0]
[1.0]
[0.05649717514124294]
[0.029069767441860465]
[1.0]
f1: >0, train: 1.000, test: 0.056
recall: >0, train: 1.000, test: 0.029
precision: >0, train: 1.000, test: 1.000
[1.0, 1.0]
[1.0, 1.0]
[1.0, 1.0]
[0.05649717514124294, 0.05649717514124294]
[0.029069767441860465, 0.029069767441860465]
[1.0, 1.0]
f1: >1, train: 1.000, test: 0.056
recall: >1, train: 1.000, test: 0.029
precision: >1, train: 1.000, test: 1.000


### Exploring Gamma Values

The gamma parameter is specific to the 'rbf' kernel in SVMs. It defines how far the influence of a single training example reaches. A low gamma means ‘far’ while a high gamma means ‘close’. Let's visualize its impact on our model's performance.

In [ ]:
# Gamma Values

# define lists to collect scores
train_f1_scores, test_f1_scores = list(), list()
train_recall_scores, test_recall_scores = list(), list()
train_prec_scores, test_prec_scores = list(), list()

# define list of C values to plot
gamma_values = [1, 0.1, 0.01, 0.001, 0.0001]

for g in gamma_values:
    # configure the model
    model = SVC(kernel='rbf', gamma=g, C=1000, class_weight='balanced', random_state=42)
    # fit the training dataset
    model.fit(X_train, y_train.values.ravel())
    # evaluate training dataset
    train_yhat = model.predict(X_train)
    train_f1 = f1_score(y_train, train_yhat)
    train_recall = recall_score(y_train, train_yhat)
    train_prec = precision_score(y_train, train_yhat)
    train_f1_scores.append(train_f1)
    print(train_f1_scores)
    train_recall_scores.append(train_recall)
    print(train_recall_scores)
    train_prec_scores.append(train_prec)
    print(train_prec_scores)
    # evaluate test dataset
    test_yhat = model.predict(x_test)
    test_f1 = f1_score(y_test, test_yhat)
    test_recall = recall_score(y_test, test_yhat)
    test_prec = precision_score(y_test, test_yhat)
    test_f1_scores.append(test_f1)
    print(test_f1_scores)
    test_recall_scores.append(test_recall)
    print(test_recall_scores)
    test_prec_scores.append(test_prec)
    print(test_prec_scores)    
    # summarize progress
    print('f1: >%d, train: %.3f, test: %.3f' % (g, train_f1, test_f1))
    print('recall: >%d, train: %.3f, test: %.3f' % (g, train_recall, test_recall))
    print('precision: >%d, train: %.3f, test: %.3f' % (g, train_prec, test_prec))


# plot of train and test scores vs tree depth
plt.plot(gamma_values, test_f1_scores, '-o', label='f1 score')
plt.plot(gamma_values, test_recall_scores, '-o', label='recall')
plt.plot(gamma_values, test_prec_scores, '-o', label='precision')
plt.legend()
plt.title("Gamma Values")
plt.xlabel("Gamma Value (C = 1000)")
plt.show()
